In [2]:
! pip install pandas
! pip install numpy
! pip install scikit-learn
! conda install -c apple tensorflow-deps
! pip install tensorflow-macos==2.12.0
! pip install tensorflow-metal==0.8.0

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - tensorflow-deps

Current channels:

  - https://conda.anaconda.org/apple/osx-64
  - https://conda.anaconda.org/apple/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


ERROR: Could not find a version that satisfies the requirement tensorflow-macos==2.9.0 (from versions: none)
ERROR: No matching distribution found for tensorflow-macos==2.9.0
ERROR: Could not find a version that satisfies the requirement tensorflow-metal

In [2]:
import pandas
import numpy

In [3]:
data = pandas.read_csv('https://gist.github.com/antonyharfield/c6e9bcf74f46645befe14f3d599c17d5/raw/51b1aaa1c43117209aa683c9650b07d9dee76805/tictactoe.csv')
data.head(10)

,board0,board1,board2,board3,board4,board5,board6,board7,board8,move0,move1,move2,move3,move4,move5,move6,move7,move8
0,-1.0,-1.0,0.0,-1.0,-1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.0,-1.0,0.0,-1.0,-1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-1.0,-1.0,0.0,-1.0,-1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-1.0,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,-1.0,-1.0,0.0,-1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,-1.0,-1.0,0.0,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-1.0,-1.0,0.0,-1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
8,-1.0,-1.0,0.0,-1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,-1.0,-1.0,0.0,-1.0,0.0,1.0,1.0,-1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
row_index = 1067

example_board_state = numpy.reshape(data.iloc[row_index, 0:9].to_numpy(), [3,3])
example_best_moves = numpy.reshape(data.iloc[row_index, 9:18].to_numpy(), [3,3])

In [5]:
print('Board state')
print(example_board_state)
print('Best moves')
print(example_best_moves)

Board state
[[-1.  1.  0.]
 [-1.  1.  1.]
 [ 0.  0. -1.]]
Best moves
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x = numpy.reshape(data.iloc[:, 0:9].to_numpy(), [-1, 3, 3])
y = numpy.reshape(data.iloc[:, 9:18].to_numpy(), [-1, 9])

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [9]:
import tensorflow-metal as tf

: 

: 

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(3, 3)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(9, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=12)

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

In [ ]:
model.add(tf.keras.layers.Reshape([3, 3]))

In [ ]:
print(numpy.around(model.predict([[[-1.0, 1.0, 0.0], [-1.0, 1.0, 1.0], [0.0, 0.0, -1.0]]])[0], 3))

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=[3,3], dtype=tf.float32)])
def wrap(input):
  input_with_batching = tf.reshape(input, [1,3,3])
  output_with_batching = model(input_with_batching)
  output = tf.reshape(output_with_batching, [3,3], name='output')
  return output

In [ ]:
output = wrap([[-1.0, 1.0, 0.0],
               [-1.0, 1.0, 1.0],
               [0.0, 0.0, -1.0]])

print(numpu.around(output, 3))

In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([wrap.get_concrete_function()])
tflite_model = converter.convert()
tflite_path = '/tmp/tictactoe.tflite'
open(tflite_path, 'wb').write(tflite_model)